## Pydex ODE Demo: Julia Version
I wanted to try recreating [pydex](https://github.com/KennedyPutraKusumo/pydex) ode demo [example](https://github.com/KennedyPutraKusumo/pydex/blob/master/examples/pydex_ode_model.ipynb) using Julia. The specific things I wanted to check out was:
* Calculate [sensitivities](https://diffeq.sciml.ai/latest/analysis/sensitivity/) using ForwardDiff automatic differentiation.
* Use [Hypatia](https://github.com/chriscoey/Hypatia.jl) conic solver with JuMP for solving the optimization problem

The following [talk](https://www.youtube.com/watch?v=DGsYIHLnR4U) by Juan Pablo Vielma and the specific slide below got me curious to try out Hypatia. 
![](figs/vielma_talk_slide.png)

But in this small example, it is hard to see the difference and it looks like Hypatia took longer than pydex demo example. But it will be interesting to try larger examples (because in Julia somtimes initial compilation may also be taking some time)

In [52]:
using Plots
using LabelledArrays
import DifferentialEquations as de
using DifferentialEquations
using Sundials
using DiffEqSensitivity
using ForwardDiff
using LinearAlgebra
using JuMP
using Hypatia

In [53]:
cA0_rng = collect(range(1.0, 5.0, length = 5))
T_rng = collect(range(273.15, 323.15, length = 5))
exp_cond = [[cA0, T] for cA0 in cA0_rng for T in T_rng]
t_rng = collect(0:20:200)

11-element Vector{Int64}:
   0
  20
  40
  60
  80
 100
 120
 140
 160
 180
 200

In [54]:
function rate!(dc, c, p, t)
    
    theta_0 = p[1]
    theta_1 = p[2]
    alpha = p[3]
    nu = p[4]
    T = p[5]
    
    cA = c[1]
    cB = c[2]
    
    r = exp(theta_0 + theta_1 * (T - 273.15)/T) * cA^alpha
    
    dc[1] = -r
    dc[2] = nu * r
end

rate! (generic function with 1 method)

In [55]:
# initial run of the ODE at a specific experimental condition to set up the ODE problem
c0 = [1.0, 0.0]
T = 300.0
p_parms = [-4.5, 2.2, 1.0, 0.5]
tspan = (0.0, 200.0)
prob = ODEProblem(rate!, c0, tspan, vcat(p_parms, T))
sol = de.solve(prob, Rodas4(), saveat = 20.0)

retcode: Success
Interpolation: 1st order linear
t: 11-element Vector{Float64}:
   0.0
  20.0
  40.0
  60.0
  80.0
 100.0
 120.0
 140.0
 160.0
 180.0
 200.0
u: 11-element Vector{Vector{Float64}}:
 [1.0, 0.0]
 [0.762974143483031, 0.1185129282584844]
 [0.5821299113834147, 0.20893504430829252]
 [0.44415180652389535, 0.27792409673805224]
 [0.33887804385509107, 0.3305609780724544]
 [0.258555354993123, 0.37072232250343834]
 [0.1972688707907377, 0.401365564604631]
 [0.15051250597534357, 0.4247437470123281]
 [0.11484163500183901, 0.4425791824990804]
 [0.08761760468079811, 0.45619119765960087]
 [0.06685148797442539, 0.46657425601278724]

In [56]:
# ODE problem put into a function so that jacobian could be evaluated with respect to function parameters
function cprofile(p_parms, c0, T)
    p = vcat(p_parms, T)
    tspan = (0.0, 200.0)
    _prob = remake(prob, u0 = c0, p = p)
    Array(de.solve(_prob, Rodas4(), saveat = 20))
end

cprofile (generic function with 1 method)

In [57]:
#csol = cprofile(p_parms, [1.0, 0.0], 300.0)
#csens = ForwardDiff.jacobian(p_parms -> cprofile(p_parms, [1.0, 0.0], 300.0), p_parms)

# Note: if there are m responses, k parameters and nT time points, the jacobian is returned as a matrix
# where # nrows = m x nT and # columns = k
# within the rows:
#   response 1 corresponds to rows 1, m+1, 2m+1, ..., m*(nT-1) + 1
#   response 2 corresponds to rows 2, m+2, 2m+2, ..., m*(nT-1) + 2 and so on    

In [58]:
csens_L = []
for (i, e) in enumerate(exp_cond)
    # this command evaluates the jacobian of responses with respect to parameters
    csens = ForwardDiff.jacobian(p_parms -> cprofile(p_parms, [e[1], 0.0], e[2]), p_parms)
    push!(csens_L, csens)
    #println(i, e)
end

In [59]:
# Generation of fim from sensitivity results
fim = []
elabel = []
for i in 1:length(exp_cond)
    for j in 1:2:22
        fim_ij = csens_L[i][j:j+1,:]' * csens_L[i][j:j+1,:]
        push!(fim, fim_ij)
        push!(elabel, vcat(exp_cond[i], t_rng[Int((j + 1)/2)]))
    end
end

In [60]:
length(fim)

275

In [61]:
# Optimization model taken from Hypatia github page. they specifically have D-optimal design as an example
#   https://github.com/chriscoey/Hypatia.jl
#
opt = Hypatia.Optimizer()
model = Model(() -> opt)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Hypatia

In [62]:
nvar = length(fim)
@variable(model, x[1:nvar] >= 0);
@constraint(model, sum(x) == 1);

In [63]:
@variable(model, hypo);
@objective(model, Max, hypo);

In [64]:
Q = sum(x[i] * fim[i] for i in 1:nvar);

In [65]:
aff = vcat(hypo, [Q[i, j] for i in 1:4 for j in 1:i]...);
@constraint(model, aff in MOI.RootDetConeTriangle(4));

In [66]:
optimize!(model)


 iter        p_obj        d_obj |  abs_gap    x_feas    z_feas |      tau       kap        mu | dir_res     prox  step     alpha
    0   2.8543e+00  -1.5344e+00 | 2.80e+02  6.29e-01  8.07e-01 | 1.00e+00  1.00e+00  1.00e+00 |
    1  -4.6108e+00  -4.5284e+00 | 4.19e+01  3.81e-01  4.88e-01 | 2.48e-01  8.29e-01  1.50e-01 | 3.6e-15  9.2e-01  co-a  8.50e-01
    2  -3.7749e+00  -4.2527e+00 | 1.26e+01  1.07e-01  1.37e-01 | 2.65e-01  1.16e-01  4.50e-02 | 1.3e-15  6.9e-01  co-a  7.00e-01
    3  -1.8831e+00  -2.1371e+00 | 8.83e+00  3.54e-02  4.55e-02 | 5.59e-01  2.77e-02  3.15e-02 | 6.2e-15  5.1e-01  co-a  3.00e-01
    4  -1.3326e+00  -1.4416e+00 | 6.15e+00  1.59e-02  2.04e-02 | 8.71e-01  2.39e-02  2.20e-02 | 8.5e-14  5.7e-01  co-a  3.00e-01
    5  -9.7946e-01  -1.0365e+00 | 4.30e+00  7.94e-03  1.02e-02 | 1.22e+00  1.34e-02  1.54e-02 | 1.3e-12  8.5e-01  co-a  3.00e-01
    6  -8.2911e-01  -8.6350e-01 | 2.99e+00  4.61e-03  5.91e-03 | 1.48e+00  7.47e-03  1.07e-02 | 1.3e-13  6.2e-01  co-a  3.00e-01


In [67]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [68]:
objective_value(model)

0.64475085348068

In [69]:
#[(elabel[i], v) for (i, v) in enumerate(value.(x)) if v >= 0.05]
for (i, v) in enumerate(value.(x))
    if v >= 0.05
        econd = elabel[i]
        effort = round(v*100,sigdigits = 4)
        println("[cA0, T, time]:$econd, Effort:$effort%")
    end
end

[cA0, T, time]:[5.0, 273.15, 60.0], Effort:16.76%
[cA0, T, time]:[5.0, 273.15, 80.0], Effort:7.415%
[cA0, T, time]:[5.0, 273.15, 200.0], Effort:27.25%
[cA0, T, time]:[5.0, 323.15, 60.0], Effort:26.4%
[cA0, T, time]:[5.0, 323.15, 160.0], Effort:22.19%
